In [1]:
# Import libraries
%reload_ext autoreload
%autoreload 2

import math
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import re
from IPython.display import display, HTML, Latex, Image, Video
from IPython.display import Markdown as md
from IPython.core.magic import register_cell_magic
from tabulate import tabulate
from sklearn.linear_model import LinearRegression
import mercury as mr
import datetime

# Allow for markdown code cells
@register_cell_magic
def markdown(line, cell):
    return md(cell.format(**globals()))

# When was last Friday?
last_friday = datetime.date.today() - pd.offsets.Week(weekday=4)

# Lab Information
defName   = "Your Name"
defCourse = "PHYS 2125 (15921)"
defSem    = "Spring 2025"
defDate   = last_friday.strftime("%Y-%m-%d")
defNum    = "10"
defName   = "Waves"
defObj    = "To determine the speed of a wave on string."
#defQ1     = "Question 1? (5 pts)"
#defQ2     = "Question 2? (5 pts)"
#defG_X    = '$M_b\; (g)$'
#defG_Y    = '$M_H\; (g)$'
#defG_T    = '$\mu_k$ as mass of block $M_b$ vs mass of hanging weight $M_H$'
columns   = ["Springs", "$M_a$ (kg)", "$x$ (m)", "$M_H$ (kg)", "$k_e$ (N/m)", "$k_e$ calculated (N/m)"];
colIdx    = 0
fileName  = 'lab10.csv'

# Mercury App
app       = mr.App(title="Lab " + defNum + " - " + defName, description="Lab report for " + defCourse + " " + defSem + " Lab " + defNum + ".")
name      = mr.Text(label="What is your name?",     value=defName, rows=1)
course    = mr.Text(label="What class are you in?", value=defCourse, rows=1)
date      = mr.Text(label="Lab Date",               value=defDate, rows=1)
labNum    = mr.Text(label="Lab Number",             value=defNum, rows=1)
labName   = mr.Text(label="Lab Name",               value=defName, rows=1)
labObj    = mr.Text(label="Objective",              value=defObj, rows=3)
#q1        = mr.Text(value="Question 1 Answer",      label=defQ1, rows=5)
#q2        = mr.Text(value="Question 2 Answer",      label=defQ2, rows=5)
#g_xaxis   = mr.Text(label="X-axis label",           value=defG_X, rows=1)
#g_yaxis   = mr.Text(label="Y-axis label",           value=defG_Y, rows=1)
#g_title   = mr.Text(label="Graph title",            value=defG_T, rows=1)
discuss   = mr.Text(value="Discussion",             label="Discuss", rows=5)
file      = mr.File(label="Data file upload",       max_file_size="10MB")

mercury.App

mercury.Text

mercury.Text

mercury.Text

mercury.Text

mercury.Text

mercury.Text

mercury.Text

mercury.File

In [2]:
%autoreload 2

if file.filepath is not None:
    df = pd.read_csv(file.filepath, names=columns, header=0).set_index(columns[colIdx])
else:
    df = pd.read_csv(fileName, names=columns, header=0).set_index(columns[colIdx])


df['error'] = df.apply(lambda row: abs(row[columns[4]] - row[columns[5]]) / row[columns[5]] * 100 
                          if df.index.get_loc(row.name) >= len(df) - 4 else np.nan, axis=1)

# Formatting
df[columns[2]] = df.apply(lambda row: f"{row[columns[2]]:.4f}", axis=1)
df[columns[3]] = df.apply(lambda row: f"{row[columns[3]]:.3f}", axis=1)
df[columns[4]] = df.apply(lambda row: f"{row[columns[4]]:.3f}", axis=1)
df[columns[5]] = df.apply(lambda row: f"{row[columns[5]]:.3f}", axis=1)

df = df.rename(columns={'error': 'Error $\%$'})

# Save this for the table
df_table = df.fillna('');

In [3]:
%%markdown

# Lab {labNum.value}: {labName.value}
Name: **{name.value}** \
Class: **{course.value}** \
Date: **{date.value}**

<p style="page-break-after:always;"></p>

## Objective

{labObj.value}


# Lab 8: Hooke's Law With Multiple Springs
Name: **Hooke's Law With Multiple Springs** \
Class: **PHYS 2125 (15921)** \
Date: **2025-03-07**

<p style="page-break-after:always;"></p>

## Objective

To determine the validity of net spring constant of experimental values with the theoretical values.


## Equipment

- (1) small A-base
- (1) long metal rod
- (1) clamp
- (1) short rod
- (1) spring set
  - (3) spring with unknown $k$ value
  - (1) 5g hook
- (1) 40g weight
- (1) 1 meter ruler

<p style="page-break-after:always;"></p>

## Theory

$F_a = M_H \cdot g$

$F_s = -kx$

$M_H g = kx$

$K = \frac{M_H g}{x}$

where

$g$ is the gravitational constant ($9.8 m/s$).

$F_a$ is the force of acceleration

$F_s$ is the spring force

$k$ is the spring constant

$x$ is the displacement of the hanging weight.

$M_h$ is the mass of the hook ($5g$)

$M_a$ is the mass added to the hook with each trial.

$M_H$ is the total mass of hanging block including the hook and added weight ($M_H = M_h + M_a$)

*Critically*, when two multiple springs are connected in series then the effective spring constant is the reciprocal of the sum of the reciprocals, e.g. with springs $k_1$ and $k_2$ in series the $k_{effective} = (1/k_1 + 1/k_2)^-1$.

In [4]:
display(Image(url='https://raw.githubusercontent.com/dundonite/Physics-Lab-2125-15921/refs/heads/main/lab4.png', width=500))

<p style="page-break-after:always;"></p>

## Procedure

The following procedure was followed.

### Initial Setup

The pendulum was constructed as follows.

1. A small cast iron A-base was placed on the table.
1. A 45cm steel rod was secured into the A-frame, raised up as much as possible to maximize the height.
1. The vertical mounting side of a steel clamp was secured at the very top of the rod.
1. A 15cm rod was attached to the horizontal side of the same steel clamp, to the far end of the smaller rod.
1. Three different springs were selected and labeled for later identification.

### Trial (completed for every combination of springs)

1. The spring(s) were suspended from the rod.
2. The 5g hook was suspended from the bottom spring.
3. A 40g weight was added to the hook, and $x$ was recorded.

<p style="page-break-after:always;"></p>

## Data

In [5]:

%autoreload 2
# Print the data table
mr.Table(data=df_table, width="25px")
#HTML(df_table.to_html(notebook=True,col_space=10))

In [6]:
%%markdown

## Calculations

See the preceeding Theory and Data table sections.

## Results

The results are evident in the data table above.

## Discussion

{discuss.value}


## Calculations

See the preceeding Theory and Data table sections.

## Results

The results are evident in the data table above.

## Discussion

Discussion
